In [1]:
import numpy as np

import rpy2.robjects as robjects
from rpy2.robjects.packages import importr
from rpy2.robjects import pandas2ri, IntVector, Formula
pandas2ri.activate()

%load_ext autoreload
%autoreload 2
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
base = importr('base')
stats = importr('stats')
ergm = importr('ergm')
network = importr('network', on_conflict="warn",
                  robject_translations={
                      'as.tibble.network': 'as_tibble_network',
                      'as_tibble.network': 'as_tibble_network'
                  }
         )

/Users/hangjianli/.pyenv/versions/3.9.2/envs/stergm/lib/python3.9/site-packages/rpy2/robjects/vectors.py:980: UserWarning: R object inheriting from "POSIXct" but without attribute "tzone".
  warnings.warn('R object inheriting from "POSIXct" but without '
/Users/hangjianli/.pyenv/versions/3.9.2/envs/stergm/lib/python3.9/site-packages/rpy2/robjects/packages_utils.py:127: UserWarning: Conflict when converting R symbols in the package "network" to Python symbols: 
-as_tibble_network -> as_tibble.network, as.tibble.network
  warn(msg)


### Load the simulated $A^t$

In [3]:
filename = '../data/res3.rds'
readRDS = robjects.r['readRDS']
res = readRDS(filename)
# res = np.array(df).astype(int)

In [66]:
resdic = dict(res.items())

In [67]:
df = resdic['nw']
df = np.array(df).astype(int)

In [68]:
resdic.get('desc')

'form: edge+mutual. diss: edge+mutual'


In [69]:
df.shape

(100, 50, 50)

## Initialize the params

In [74]:
num_nodes = df[0].shape[0]
# n = 20
T = len(df)
# num_changepts = 5

p_pos = 2 # edge + mutual
p_neg = 2 # edge + mutual
p = p_pos + p_neg

np.random.seed(42)
theta = np.random.normal(0, 0.5, size= (T , p)) 
z = np.copy(theta) 
u = np.zeros((T, p)) 


form_terms = ['edges', 'mutual']
diss_terms = ['edges', 'mutual']

/Users/hangjianli/.pyenv/versions/3.7.4/envs/leetcode/lib/python3.7/site-packages/rpy2/robjects/packages_utils.py:127: UserWarning: Conflict when converting R symbols in the package "network" to Python symbols: 
-as_tibble_network -> as_tibble.network, as.tibble.network
  warn(msg)
/Users/hangjianli/.pyenv/versions/3.7.4/envs/leetcode/lib/python3.7/site-packages/rpy2/robjects/vectors.py:980: UserWarning: R object inheriting from "POSIXct" but without attribute "tzone".
  warnings.warn('R object inheriting from "POSIXct" but without '


In [75]:
init = {
    'theta': np.copy(theta),
    'z': z,
    'u': u
}

In [79]:
init['theta']

array([[-8.67099841, -2.29890329,  8.32236463, -1.57590162],
       [-0.95769752, -1.72284883,  2.177566  , -2.53238916],
       [-0.97408992, -2.01080285,  2.23920503, -1.9825493 ],
       [-1.1838951 , -1.70182544,  1.94354064, -1.8870211 ],
       [-0.99753522, -1.99517233,  1.99909886, -1.74335006],
       [-1.26348028, -1.65566019,  2.18024622, -1.74381773],
       [-0.8906448 , -2.11145191,  2.05345874, -2.06875144],
       [-1.24798147, -1.80445829,  2.04717475, -1.95975169],
       [-1.06791539, -1.97544271,  2.16205659, -2.30328387],
       [-1.05806925, -1.97710134,  2.19784611, -2.32194455],
       [-1.30052871, -1.65422795,  2.02213895, -1.98361622],
       [-1.0707358 , -2.07573666,  2.11451147, -2.46145596],
       [-0.94466022, -2.21722144,  2.10434706, -2.41735501],
       [-0.8542176 , -2.14683782,  2.04495897, -1.81694517],
       [-1.34477662, -1.6443523 ,  2.12575954, -2.21591895],
       [-1.03677441, -2.18419003,  2.26285694, -1.65519206],
       [-1.03086522, -1.

In [76]:
from mple_learn import stergmGraph
sim1 = stergmGraph(
    X = df,
    form_terms=form_terms,
    diss_terms=diss_terms,
    lam=[1],
    max_steps=50,
    newton_max_steps=100,
    admm_alpha=1,
    converge_tol=1e-5,
    verbose=1
)

In [100]:
theta_res, z, u, c = sim1.mple(initial_values=init, solver='newton', tau_inc=2, m=10)

In [80]:
init['theta'].shape

(100, 4)

In [99]:
np.arange(99)[np.linalg.norm(np.diff(init['theta'], axis=0), ord=2, axis=1) > 4]

array([ 0, 19, 24, 59, 79])

In [85]:
np.round(init['theta'],2)

array([[-8.67, -2.3 ,  8.32, -1.58],
       [-0.96, -1.72,  2.18, -2.53],
       [-0.97, -2.01,  2.24, -1.98],
       [-1.18, -1.7 ,  1.94, -1.89],
       [-1.  , -2.  ,  2.  , -1.74],
       [-1.26, -1.66,  2.18, -1.74],
       [-0.89, -2.11,  2.05, -2.07],
       [-1.25, -1.8 ,  2.05, -1.96],
       [-1.07, -1.98,  2.16, -2.3 ],
       [-1.06, -1.98,  2.2 , -2.32],
       [-1.3 , -1.65,  2.02, -1.98],
       [-1.07, -2.08,  2.11, -2.46],
       [-0.94, -2.22,  2.1 , -2.42],
       [-0.85, -2.15,  2.04, -1.82],
       [-1.34, -1.64,  2.13, -2.22],
       [-1.04, -2.18,  2.26, -1.66],
       [-1.03, -1.76,  1.95, -1.88],
       [-1.03, -2.11,  1.91, -2.33],
       [-0.74, -2.09,  2.13, -2.75],
       [-1.05, -1.95,  1.91, -1.82],
       [ 1.58,  1.36,  1.09,  2.79],
       [ 1.8 ,  0.88,  1.01,  2.02],
       [ 2.16,  1.16,  1.15,  1.86],
       [ 2.12,  1.18,  1.41,  1.83],
       [ 1.79,  1.09,  0.71,  2.5 ],
       [-0.77,  4.38,  1.31,  1.92],
       [-0.51,  3.74,  1.02,  2.15],
 

In [86]:
resdic['coefs_pos']
resdic['coefs_neg']

array([[-1.,  1.,  2., -3.,  1.],
       [-2.,  2.,  1., -1.,  3.]])

array([[ 2.07944154,  1.09861229, -0.69314718,  0.69314718,  2.30258509],
       [-2.        ,  2.        , -1.        , -1.        ,  3.        ]])

In [87]:
np.append(resdic['coefs_pos'], resdic['coefs_neg'], axis=0).T

array([[-1.        , -2.        ,  2.07944154, -2.        ],
       [ 1.        ,  2.        ,  1.09861229,  2.        ],
       [ 2.        ,  1.        , -0.69314718, -1.        ],
       [-3.        , -1.        ,  0.69314718, -1.        ],
       [ 1.        ,  3.        ,  2.30258509,  3.        ]])

### Test Newton's method

In [9]:
hess_f = sim1.theta_update_hess_f(theta)
grad_f = sim1.theta_update_grad_f(theta, z, u)

In [39]:
theta_res = sim1.theta_update(init['theta'], z, u)

[INFO]	Inner Step # 0. Current diff is 1.0000001
[INFO] Loss = 33092.597021391324
max of mu is 0.9996489878379565
[INFO]	Inner Step # 1. Current diff is 9.617097546986221
[INFO] Loss = 11305.777727932968
max of mu is 0.9632169359769646
[INFO]	Inner Step # 2. Current diff is 0.2595090612892723
[INFO] Loss = 11861.002038092121
max of mu is 0.7239483473007992
[INFO]	Inner Step # 3. Current diff is 0.459800049101596
[INFO] Loss = 11586.37767534945
max of mu is 0.7829629395469192
[INFO]	Inner Step # 4. Current diff is 0.025880145267771548
[INFO] Loss = 11580.147453216698
max of mu is 0.788276237493941
[INFO]	Inner Step # 5. Current diff is 8.945294012274224e-05


In [140]:
theta_res

array([[-2.39290964,  0.60903965,  0.4352056 , -1.80261384],
       [-1.8178356 , -1.55117841,  2.27144208, -0.46653777],
       [-4.56820457, -3.919792  , -1.51145279,  2.52212431],
       [-4.73860946, -3.93336226,  1.89421578, -1.48659024],
       [-0.36294804,  0.09303787,  1.08081491,  1.98301898],
       [ 1.39896977,  1.27439511,  0.42027876,  1.07446576],
       [ 2.32866375,  2.83051687,  0.46795194,  2.46968428],
       [ 3.03979677,  0.28846171,  2.88408828,  0.77524795],
       [-1.99453383, -0.85852602,  2.52752802,  1.03164035],
       [-3.93030243,  1.13600796,  3.81299235,  1.31769738],
       [ 2.78059519,  0.91625975,  0.279195  , -0.67474795],
       [ 4.18951476,  2.79440973, -4.28815829, -3.36843649],
       [-0.167136  ,  1.79699814, -3.21230955,  0.16659357],
       [ 3.28416616,  1.59893526, -4.20601004,  2.47416368],
       [ 5.00130672, -0.09500702, -1.49619046, -2.11945833]])

### Test fused lasso

In [48]:
import matlab.engine
eng = matlab.engine.start_matlab()
eng.addpath(eng.genpath('../gfl/'), nargout=0)

### Test mple

In [ ]:
theta_res